**Imports**

In [ ]:
#FINANCIAL LIBRARIES
import FundamentalAnalysis as fa
import talib as ta

#OTHER LIBRARIES
import import_ipynb
import pandas as pd
import numpy as np
from datetime import date
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import datetime
import UTILS as utils

In [ ]:
api_key = "Here would go my personal API key"

----------

**Define tickers**

In [ ]:
#We'll reserve the variables name containing '_ticker_...' for the tickers inputed by the user

#EXAMPLES
_ticker_1 = 'AMZN'
_ticker_2 = 'MCD'

In [ ]:
tickers = [globals()[ticker] for ticker in list(globals().keys()) if "_ticker_" in ticker]

In [ ]:
tickers

--------

**Load the needed data for each ticker**

In [ ]:
#Today date
today = date.today()

In [ ]:
def load_stock_data(ticker, dict_companies):
    
    profile = dict_companies[ticker]['_Company__profile']

    stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

    ipo_year = int(profile.loc['ipoDate', 0][:4])
    stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin=f"{str(ipo_year+1)}-01-01", end=str(today.strftime("%Y-%m-%d"))).iloc[::-1]
    
    return stock_data, stock_data_detailed

In [ ]:
tickers_stock_data = dict()
for _ticker in tickers:
    tickers_stock_data[_ticker] = load_stock_data(_ticker, dict_companies)

-----------------

**Evolution function definition for price, volume and daily returns plots**

In [ ]:
def plot_evolution(ticker, stock_data_detailed, evolution_var='price', line_chart=False):
    
    contempled_evolution_vars = ['price', 'daily returns', 'volume']
    if evolution_var not in contempled_evolution_vars: raise Exception(f'You can not visualize {evolution_var}')
    
    
    if isinstance(ticker, list) and isinstance(stock_data_detailed, list) and (len(ticker)==len(stock_data_detailed)):
        last_ipo = max([pd.to_datetime(sd.index[0], format='%Y-%m-%d') for sd in stock_data_detailed])
        for sd in range(len(stock_data_detailed)):
            stock_data_detailed[sd].index = pd.to_datetime(stock_data_detailed[sd].index)
            stock_data_detailed[sd] = stock_data_detailed[sd].loc[stock_data_detailed[sd].index > last_ipo] 
            stock_data_detailed[sd] = stock_data_detailed[sd].reset_index().rename(columns={'index':'date'})
            stock_data_detailed[sd]['ticker'] = ticker[sd]
            if evolution_var == 'daily returns': stock_data_detailed[sd]['daily_returns'] = stock_data_detailed[sd]['adjClose'].pct_change()
            if sd == 0: concat_df = stock_data_detailed[sd]
            else: concat_df = pd.concat([concat_df, stock_data_detailed[sd]], axis=0)
                
        if evolution_var == 'price':
            fig = px.line(concat_df, 
                          x='date', 
                          y='adjClose',
                          color='ticker',
                          title=f'Line chart of the stock price evolution for {", ".join(ticker)}',
                          labels=dict(date="date", adjClose="price($)"))
            
        elif evolution_var == 'daily returns':
            fig = px.line(concat_df, 
                          x='date', 
                          y='daily_returns',
                          color='ticker',
                          title=f'Line chart of the daily returns evolution for {", ".join(ticker)}',
                          labels=dict(date="date", daily_returns="daily returns($)"))            
            
        else:
            fig = px.line(concat_df, 
                          x='date', 
                          y='volume',
                          color='ticker',
                          title=f'Line chart of the volume of trade evolution for {", ".join(ticker)}',
                          labels=dict(date="date", volume="volume($)"))
        fig.show()
        
    else:
        if not line_chart and evolution_var=='price':
            candlestick = go.Candlestick(x=stock_data_detailed.index,
                                    open=stock_data_detailed['open'],
                                    high=stock_data_detailed['high'],
                                    low=stock_data_detailed['low'],
                                    close=stock_data_detailed['adjClose'])
            fig = go.Figure(data=[candlestick])
            fig.update_layout(title_text=f'Candlestick chart of the stock price evolution for {ticker}',
                              xaxis_rangeslider_visible=False,
                              yaxis_title='price($)')
        else:
            stock_data_detailed = stock_data_detailed.reset_index().rename(columns={'index':'date'})
            if evolution_var == 'price':
                fig = px.line(stock_data_detailed, 
                              x='date', 
                              y='adjClose', 
                              title=f'Line chart of the stock price evolution for {ticker}',
                              labels=dict(date="date", adjClose="price($)"))
            elif evolution_var == 'daily returns':
                stock_data_detailed['daily_returns'] = stock_data_detailed['adjClose'].pct_change()
                fig = px.line(stock_data_detailed, 
                              x='date', 
                              y='daily_returns', 
                              title=f'Line chart of the daily returns evolution for {ticker}',
                              labels=dict(date="date", daily_returns="daily returns($)"))
            else:
                fig = px.line(stock_data_detailed, 
                              x='date', 
                              y='volume', 
                              title=f'Line chart of the volume of trade evolution for {ticker}',
                              labels=dict(date="date", volume="volume($)"))
        fig.show()

--------

**Stock price**

Consider to take a look at *adjclose* while analysing the stock price because is mostly used for historical returns. 
Also, the adjclose changes where the company has had dividend distributions and splits. 
On the other hand, close price does not take into account dividend distributions and splits.

*Visualization*

In [ ]:
for _ticker in tickers:
    plot_evolution(_ticker, tickers_stock_data[_ticker][1], 'price', True)

In [ ]:
plot_evolution(tickers, [tickers_stock_data[_ticker][1] for _ticker in tickers], 'price', True)

*Statistics*

In [ ]:
for _ticker in tickers:
    print(f'\n\nStatistics for {_ticker} stock data')
    display(tickers_stock_data[_ticker][0].describe())

----

**Daily returns**

*Line chart*

In [ ]:
for _ticker in tickers:
    plot_evolution(_ticker, tickers_stock_data[_ticker][1], 'daily returns', True)

In [ ]:
plot_evolution(tickers, [tickers_stock_data[_ticker][1] for _ticker in tickers], 'daily returns', True)

*Histogram*

In [ ]:
daily_returns = dict()
for _ticker in tickers:
    daily_returns[_ticker] = tickers_stock_data[_ticker][1]['adjClose'].pct_change()
    plt.figure(figsize=(14,7))
    plt.hist(daily_returns[_ticker].dropna(), bins=100, alpha=0.6, color='b') # Drop NaN
    plt.title(f"Histogram of daily returns for {_ticker}", fontsize=16)
    plt.axvline(daily_returns[_ticker].mean(), color='r', linestyle='dashed', linewidth=2) # Shows the average line
    plt.xlabel("Daily returns", fontsize=12)
    plt.ylabel("Freq of daily return", fontsize=12)
    plt.show()
    print()

---------

**Volume traded**

*Line chart*

In [ ]:
for _ticker in tickers:
    plot_evolution(_ticker, tickers_stock_data[_ticker][1], 'volume', True)

In [ ]:
plot_evolution(tickers, [tickers_stock_data[_ticker][1] for _ticker in tickers], 'volume', True)

-------

**Log returns and volatility**

In [ ]:
def returns_volatility_evolution(ticker, stock_data_detailed):
    log_returns_volatility = dict()
    daily_log_returns = np.log((stock_data_detailed['adjClose']/stock_data_detailed['adjClose'].shift()).astype('float64'))
    volatility = daily_log_returns.std()*252**.5


    plt.figure(figsize=(14,7))
    plt.hist(daily_log_returns.dropna(), bins=50, alpha=0.6, color='b')
    plt.title(f"{_ticker} volatility: {str(round(volatility, 4)*100)}%", fontsize=16)
    plt.axvline(daily_log_returns.mean(), color='r', linestyle='dashed', linewidth=2) # Shows the average line
    plt.xlabel("Daily log return", fontsize=12)
    plt.ylabel("Freq of daily log return", fontsize=12)
    plt.show()
    
    return daily_log_returns, volatility

In [ ]:
log_returns_volatility = dict()
for _ticker in tickers:
    log_returns_volatility[_ticker] = list(returns_volatility_evolution(_ticker, tickers_stock_data[_ticker][1]))

---------

**Dividends evolution visualization**

In [ ]:
def dividend_evolution(ticker, dividends):
    if isinstance(dividends, pd.DataFrame):
        dividends.index = pd.to_datetime(dividends.index)
        dividends = dividends.reset_index().rename(columns={'index':'date'})
        fig = px.line(dividends, 
                      x='date', 
                      y='adjDividend', 
                      title=f'Line chart of the dividends evolution for {ticker}',
                      labels=dict(date="date", adjDividend="amount($)"))
        fig.show()
    else:
        print(f'{ticker} DOES NOT DISTRIBUTE DIVIDENDS')

In [ ]:
for _ticker in tickers:
    dividend_evolution(_ticker, dict_companies[_ticker]['_Company__dividends'])

--------

**Additional technical indicators**

In [ ]:
#SIMPLE AND EXPONENTIAL MOVING AVERAGES

def moving_average(ticker, stock_data_detailed, type_ma='simple', period=30):

    if type_ma == 'simple':
        ma_df = pd.DataFrame(ta.MA(stock_data_detailed['adjClose'], timeperiod=period), columns=['value']).reset_index().rename(columns={'index':'date'})
    elif type_ma == 'exponential':
        ma_df = pd.DataFrame(ta.EMA(stock_data_detailed['adjClose'], timeperiod=period), columns=['value']).reset_index().rename(columns={'index':'date'})
        
    ma_df['line'] = 'ma'
    price_df = pd.DataFrame(stock_data_detailed['adjClose']).reset_index().rename(columns={'index':'date', 'adjClose':'value'})
    price_df['line'] = 'price'
    concat_df = pd.concat([ma_df, price_df])
    fig = px.line(concat_df, 
                  x='date', 
                  y='value',
                  color='line',
                  title=f'{type_ma.capitalize()} Moving Average for {ticker}',
                  labels=dict(date="date", value="amount($)"))

    fig.show()

In [ ]:
for _ticker in tickers:
    moving_average(_ticker, tickers_stock_data[_ticker][1], 'simple', 30)

--------

In [ ]:
#RELATIVE STRENGHT INDEX

def relative_strenght_index(ticker, stock_data_detailed, period=5):
    
    rsi_df = pd.DataFrame(ta.RSI(stock_data_detailed['adjClose'], timeperiod=period), columns=['rsi'])
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
    fig.add_trace(go.Candlestick(x=stock_data_detailed.index,
                                   open=stock_data_detailed['open'],
                                   high=stock_data_detailed['high'],
                                   low=stock_data_detailed['low'],
                                   close=stock_data_detailed['adjClose']), row=1, col=1)

    fig.add_trace(go.Scatter(x=rsi_df.index,
                             y=rsi_df['rsi'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=2, col=1)

    fig.update_yaxes(range=[-10, 110], row=2, col=1)
    fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=1)
    fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=1)

    fig.add_hline(y=30, col=1, row=2, line_color='#336699', line_width=1, line_dash='dash')
    fig.add_hline(y=70, col=1, row=2, line_color='#336699', line_width=1, line_dash='dash')

    layout = go.Layout(
        title=f'Relative Strenght Index for {ticker}',
        plot_bgcolor='#efefef',
        xaxis=dict(
            rangeslider=dict(
            visible=False
            )
        )
    )
    fig.update_layout(layout)
    fig.show()

In [ ]:
for _ticker in tickers:
    relative_strenght_index(_ticker, tickers_stock_data[_ticker][1], period=5)

-------------

In [ ]:
#BOLLINGER BANDS

def bollinger_bands(ticker, stock_data_detailed, timeperiod=20, nbdevup=2, nbdevdn=2): #nbdev = non-biased standard deviations from the mean
    
    adjclose_df = pd.DataFrame(stock_data_detailed[['adjClose']].copy().rename(columns={'adjClose': 'price'}))
    ub_df = pd.DataFrame(index=adjclose_df.index)
    mb_df = pd.DataFrame(index=adjclose_df.index)
    lb_df = pd.DataFrame(index=adjclose_df.index)
    ub_df['price'], mb_df['price'], lb_df['price'] = ta.BBANDS(stock_data_detailed['adjClose'], timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=0)
    adjclose_df['line'] = 'adjclose'
    ub_df['line'] = 'upper band'
    mb_df['line'] = 'middle band'
    lb_df['line'] = 'lower band'
    bollinger_dfs = [adjclose_df, ub_df, mb_df, lb_df]
    concat_bollinger_df = pd.concat(bollinger_dfs).reset_index().rename(columns={'index':'date'})
    fig = px.line(concat_bollinger_df, 
                  x='date', 
                  y='price',
                  color='line',
                  title=f'Bollingers Bands for {_ticker}',
                  labels=dict(date="date", price="price($)"))
    fig.show()
    

In [ ]:
for _ticker in tickers:
    bollinger_bands(_ticker, tickers_stock_data[_ticker][1], 20, 2, 2)

----------------------

In [ ]:
#ON BALANCE VOLUME 

def on_balance_volume(ticker, stock_data_detailed):
    obv_df = pd.DataFrame(ta.OBV(stock_data_detailed['adjClose'], stock_data_detailed['volume']), columns=['obv'])
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
    fig.add_trace(go.Candlestick(x=stock_data_detailed.index,
                                       open=stock_data_detailed['open'],
                                       high=stock_data_detailed['high'],
                                       low=stock_data_detailed['low'],
                                       close=stock_data_detailed['adjClose']), row=1, col=1)

    fig.add_trace(go.Scatter(x=obv_df.index,
                                 y=obv_df['obv'],
                                 line=dict(color='#ff9900', width=0.5),
                                 showlegend=False), row=2, col=1)

    fig.update_yaxes(range=[obv_df.min()-(0.05*abs(obv_df.min())), obv_df.max()+(0.05*abs(obv_df.min()))], row=2, col=1)

    layout = go.Layout(
            title=f'On Balance Volume for {ticker}',
            plot_bgcolor='#efefef',
            xaxis=dict(
                rangeslider=dict(
                visible=False
                )
            )
        )
    fig.update_layout(layout)
    fig.show()

In [ ]:
for _ticker in tickers:
    on_balance_volume(_ticker, tickers_stock_data[_ticker][1])

--------------

In [ ]:
#CHAIKIN A/D OSCILLATOR 

def chaikin_ad_oscillator(ticker, stock_data_detailed, fast_period=3, slow_period=10):
   
    ad_df = pd.DataFrame(ta.ADOSC(stock_data_detailed['high'], stock_data_detailed['low'], stock_data_detailed['adjClose'], stock_data_detailed['volume'], fastperiod=fast_period, slowperiod=slow_period), columns=['ad'])
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
    fig.add_trace(go.Candlestick(x=stock_data_detailed.index,
                                   open=stock_data_detailed['open'],
                                   high=stock_data_detailed['high'],
                                   low=stock_data_detailed['low'],
                                   close=stock_data_detailed['adjClose']), row=1, col=1)

    fig.add_trace(go.Scatter(x=ad_df.index,
                             y=ad_df['ad'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=2, col=1)

    fig.update_yaxes(range=[ad_df.min()-(0.05*abs(ad_df.min())), ad_df.max()+(0.05*abs(ad_df.min()))], row=2, col=1)

    fig.add_hline(y=0, col=1, row=2, line_color='#336699', line_width=1, line_dash='dash')

    layout = go.Layout(
        title=f'Chaikin A/D Oscillator for {ticker}',
        plot_bgcolor='#efefef',
        xaxis=dict(
            rangeslider=dict(
            visible=False
            )
        )
    )
    fig.update_layout(layout)
    fig.show()

In [ ]:
for _ticker in tickers:
    chaikin_ad_oscillator(_ticker, tickers_stock_data[_ticker][1], fast_period=3, slow_period=10)

----------

In [ ]:
#AVERAGE DIRECTIONAL MOVEMENT INDEX

def average_directional_movement_index(ticker, stock_data_detailed, timeperiod=14): #14 periods are typically used
    
    
    adx_df = pd.DataFrame(ta.ADX(stock_data_detailed['high'], stock_data_detailed['low'], stock_data_detailed['adjClose'], timeperiod=timeperiod), columns=['adx'])
    posdi_df = pd.DataFrame(ta.PLUS_DI(stock_data_detailed['high'], stock_data_detailed['low'], stock_data_detailed['adjClose'], timeperiod=timeperiod), columns=['positive_di'])
    negdi_df = pd.DataFrame(ta.MINUS_DI(stock_data_detailed['high'], stock_data_detailed['low'], stock_data_detailed['adjClose'], timeperiod=timeperiod), columns=['negative_di'])    
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                        row_heights=[0.75, 0.25, 0.25, 0.25],
                        vertical_spacing = 0.06,
                        subplot_titles=['', '+DI', '-DI', 'ADX'])
    fig.add_trace(go.Candlestick(x=stock_data_detailed.index,
                                   open=stock_data_detailed['open'],
                                   high=stock_data_detailed['high'],
                                   low=stock_data_detailed['low'],
                                   close=stock_data_detailed['adjClose']), row=1, col=1)

    fig.add_trace(go.Scatter(x=posdi_df.index,
                             y=posdi_df['positive_di'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=2, col=1)
    fig.add_trace(go.Scatter(x=negdi_df.index,
                             y=negdi_df['negative_di'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=3, col=1)
    fig.add_trace(go.Scatter(x=adx_df.index,
                             y=adx_df['adx'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=4, col=1)

    fig.update_yaxes(range=[posdi_df.min()-(0.05*abs(posdi_df.min())), posdi_df.max()+(0.05*abs(posdi_df.min()))], row=2, col=1)
    fig.update_yaxes(range=[negdi_df.min()-(0.05*abs(negdi_df.min())), negdi_df.max()+(0.05*abs(negdi_df.min()))], row=3, col=1)
    fig.update_yaxes(range=[adx_df.min()-(0.05*abs(adx_df.min())), adx_df.max()+(0.05*abs(adx_df.min()))], row=4, col=1)

    layout = go.Layout(
        autosize=False,
        width=900,
        height=1000,
        title=f'Average Directional Movement Index for {ticker}',
        plot_bgcolor='#efefef',
        xaxis=dict(
            rangeslider=dict(
            visible=False
            )
        )
    )
    fig.update_layout(layout)
    fig.show()


In [ ]:
for _ticker in tickers:
    average_directional_movement_index(_ticker, tickers_stock_data[_ticker][1], timeperiod=14)

----------

In [ ]:
#AROON OSCILLATOR

def aroon_oscillator(ticker, stock_data_detailed, timeperiod=25):  # Typically uses 25 periods
    aronsc_df =  pd.DataFrame(ta.AROONOSC(tickers_stock_data[_ticker][1]['high'], tickers_stock_data[_ticker][1]['low'], timeperiod=25), columns=['aroon_osc'])
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_width=[0.25, 0.75])
    fig.add_trace(go.Candlestick(x=stock_data_detailed.index,
                                   open=stock_data_detailed['open'],
                                   high=stock_data_detailed['high'],
                                   low=stock_data_detailed['low'],
                                   close=stock_data_detailed['adjClose']), row=1, col=1)

    fig.add_trace(go.Scatter(x=aronsc_df.index,
                             y=aronsc_df['aroon_osc'],
                             line=dict(color='#ff9900', width=0.5),
                             showlegend=False), row=2, col=1)

    fig.update_yaxes(range=[aronsc_df.min()-(0.05*abs(aronsc_df.min())), aronsc_df.max()+(0.05*abs(aronsc_df.min()))], row=2, col=1)

    fig.add_hline(y=0, col=1, row=2, line_color='#336699', line_width=1, line_dash='dash')

    layout = go.Layout(
        title=f'Aroon Oscillator for {ticker}',
        plot_bgcolor='#efefef',
        xaxis=dict(
            rangeslider=dict(
            visible=False
            )
        )
    )
    fig.update_layout(layout)
    fig.show()

In [ ]:
for _ticker in tickers:
    aroon_oscillator(_ticker, tickers_stock_data[_ticker][1], timeperiod=25)